In [2]:
import numpy as np
import pandas as pd
import plotly.express as pltly

## Q: First: explore the data
Review the unstructured csv files and answer the following questions with code that supports your conclusions:
- Are there any data quality issues present?
- Are there any fields that are challenging to understand?

In [3]:
# Import all data files
users = pd.read_csv('./USER_TAKEHOME.csv')
txns = pd.read_csv('./TRANSACTION_TAKEHOME.csv')
products = pd.read_csv('./PRODUCTS_TAKEHOME.csv')

In [13]:
users.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female


In [14]:
txns.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


In [15]:
products.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11


### Exploring Users Dataset

In [16]:
users.isna().sum()

ID                  0
CREATED_DATE        0
BIRTH_DATE       3675
STATE            4812
LANGUAGE        30508
GENDER           5892
dtype: int64

- ID and CREATED_DATE have no missing values. Could be candidate keys.
- The dataset has notable gaps in demographic fields like BIRTH_DATE, STATE, LANGUAGE, GENDER, suggesting incomplete user profiles

In [30]:
# Get a list of unique states in the users dataset
unique_states = users[users['STATE'].notna()]['STATE'].unique()

# Number of unique states
print(len(unique_states))

# List of unique states
print(users[users['STATE'].notna()]['STATE'].unique())

52
['CA' 'PA' 'FL' 'NC' 'NY' 'IN' 'OH' 'TX' 'NM' 'PR' 'CO' 'AZ' 'RI' 'MO'
 'NJ' 'MA' 'TN' 'LA' 'NH' 'WI' 'IA' 'GA' 'VA' 'DC' 'KY' 'SC' 'MN' 'WV'
 'DE' 'MI' 'IL' 'MS' 'WA' 'KS' 'CT' 'OR' 'UT' 'MD' 'OK' 'NE' 'NV' 'AL'
 'AK' 'AR' 'HI' 'ME' 'ND' 'ID' 'WY' 'MT' 'SD' 'VT']


- There are 52 unique states, when in fact there are only 50 states in USA
- Upon closer investigation, we can see that 'DC' and 'PR' are also included which are separate regions or territories

In [31]:
users['LANGUAGE'].value_counts()

LANGUAGE
en        63403
es-419     6089
Name: count, dtype: int64

- The users dataset contains two unique language entries: en and es
- While the exact context of these language codes is not explicitly defined in the dataset, it can be assumed that en likely represents English and es represents Español (Spanish)
- These codes may indicate the language in which the data was collected or the preferred language of the users

In [35]:
users['GENDER'].value_counts()

GENDER
female                    64240
male                      25829
transgender                1772
prefer_not_to_say          1350
non_binary                  473
unknown                     196
not_listed                  180
Non-Binary                   34
not_specified                28
My gender isn't listed        5
Prefer not to say             1
Name: count, dtype: int64

- Entries like {"non_binary" & "Non-Binary"} and {"Prefer not to say" & "prefer_not_to_say"} should be standardized.
- It is unclear what labels such as "unknown" and "not_specified" signify and could overlap with other categories
- Entries like "My gender isn't listed" (5) could be consolidated with other Categories for analysis

### Exploring Products Dataset

In [18]:
products.isna().sum()

CATEGORY_1         111
CATEGORY_2        1424
CATEGORY_3       60566
CATEGORY_4      778093
MANUFACTURER    226474
BRAND           226472
BARCODE           4025
dtype: int64

- The Barcode of a product is intended to be a foreign key in the Transactions table.
- There are 4,025 products without a barcode, making it impossible to uniquely identify these products.
- As a result, these products cannot be used in transactions due to the absence of a unique identifier.

In [21]:
products[products['CATEGORY_1'].isna()]

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
5184,NaN,NaN,NaN,NaN,KEURIG DR PEPPER,POLAR,7.153711e+11
15048,NaN,NaN,NaN,NaN,THE COCA-COLA COMPANY,COCA-COLA,1.963655e+11
25313,NaN,NaN,NaN,NaN,MOLSONCOORS,COORS LIGHT,1.981811e+11
35604,NaN,NaN,NaN,NaN,THE COCA-COLA COMPANY,COCA-COLA,1.969831e+11
43214,NaN,NaN,NaN,NaN,KEURIG DR PEPPER,POLAR,6.443761e+11
...,...,...,...,...,...,...,...
815098,NaN,NaN,NaN,NaN,MOLSONCOORS,COORS LIGHT,1.981811e+11
828059,NaN,NaN,NaN,NaN,PEPSICO,CHEETOS,2.840072e+10
838895,NaN,NaN,NaN,NaN,THE COCA-COLA COMPANY,COCA-COLA,1.969831e+11
841164,NaN,NaN,NaN,NaN,MOLSONCOORS,COORS LIGHT,1.981811e+11


- Category_1 has 111 missing values. For these records, all other category fields are also null, indicating 111 products with no category information assigned

In [16]:
products['CATEGORY_1'].nunique()

27

In [11]:
products['CATEGORY_2'].nunique()

121

In [12]:
products['CATEGORY_3'].nunique()

344

In [13]:
products['CATEGORY_4'].nunique()

127

In [14]:
products['MANUFACTURER'].nunique()

4354

In [15]:
products['BRAND'].nunique()

8122

### Exploring Transactions Dataset

In [30]:
txns.isna().sum()

RECEIPT_ID           0
PURCHASE_DATE        0
SCAN_DATE            0
STORE_NAME           0
USER_ID              0
BARCODE           5762
FINAL_QUANTITY       0
FINAL_SALE           0
dtype: int64

- The Barcode field in the Transaction table serves as a foreign key to uniquely identify a product in the Product table.
- There are 5,762 transactions with a missing barcode, preventing these transactions from being linked to a specific product.

In [27]:
txns['STORE_NAME'].nunique()

954

In [29]:
txns['FINAL_QUANTITY'].unique()

array(['1.00', 'zero', '2.00', '3.00', '4.00', '4.55', '2.83', '2.34',
       '0.46', '7.00', '18.00', '12.00', '5.00', '2.17', '0.23', '8.00',
       '1.35', '0.09', '2.58', '1.47', '16.00', '0.62', '1.24', '1.40',
       '0.51', '0.53', '1.69', '6.00', '2.39', '2.60', '10.00', '0.86',
       '1.54', '1.88', '2.93', '1.28', '0.65', '2.89', '1.44', '2.75',
       '1.81', '276.00', '0.87', '2.10', '3.33', '2.54', '2.20', '1.93',
       '1.34', '1.13', '2.19', '0.83', '2.61', '0.28', '1.50', '0.97',
       '0.24', '1.18', '6.22', '1.22', '1.23', '2.57', '1.07', '2.11',
       '0.48', '9.00', '3.11', '1.08', '5.53', '1.89', '0.01', '2.18',
       '1.99', '0.04', '2.25', '1.37', '3.02', '0.35', '0.99', '1.80',
       '3.24', '0.94', '2.04', '3.69', '0.70', '2.52', '2.27'],
      dtype=object)

### Closed-ended questions:


#1 What are the top 5 brands by receipts scanned among users 21 and over?


In [3]:
query = """
-- CTE 1: Filter users who have had an account for at least six months.
WITH Filtered_Users AS (
    SELECT 
        ID AS USER_ID  
    FROM Users
    WHERE DATEDIFF('2025-01-17', CREATED_DATE) >= 180  -- Only include users with accounts older than 180 days
),

-- CTE 2: Filter transactions from the users identified in Step 1.
Valid_Transactions AS (
    SELECT 
        t.BARCODE,                 
        t.FINAL_SALE AS SALE_AMOUNT
    FROM Transactions t
    WHERE t.USER_ID IN (SELECT USER_ID FROM Filtered_Users) -- Only consider transactions from filtered users
    AND t.FINAL_SALE IS NOT NULL  -- Exclude transactions where the sale amount is NULL
),

-- CTE 3: Aggregate sales data by brand.
Brand_Sales AS (
    SELECT 
        p.BRAND,                  
        SUM(vt.SALE_AMOUNT) AS TOTAL_SALES -- Total sales amount for the brand
    FROM Valid_Transactions vt
    INNER JOIN Products p
        ON vt.BARCODE = p.BARCODE  -- Match transactions to products using the barcode
	WHERE p.BRAND IS NOT NULL      -- Exclude NULL brand values
    GROUP BY p.BRAND               -- Group sales data by brand
)

-- CTE 4: Fetch the top 5 brands based on total sales.
SELECT 
    BRAND,           
    TOTAL_SALES     
FROM Brand_Sales
ORDER BY TOTAL_SALES DESC  -- Sort brands by total sales in descending order
LIMIT 5;                   -- Limit the result to the top 5 brands
"""

In [4]:
from pandasql import sqldf
# Define a function to execute SQL queries (recommended for reusability)
def pysqldf(q):
    return sqldf(q, globals())

In [ ]:
result_df = pysqldf(query)
print(result_df)